In [52]:
from dl_func import *
from sits_func import *
from res_func import *
from tensorflow.python.lib.io import file_io


In [13]:
train_file = r'F:\acoca\research\gee\dataset\AMZ\comparison\input\fcTS_train_ssize10_tyear2002_LCfinalmapv6_LCProp2.csv'
test_file = r'F:\acoca\research\gee\dataset\AMZ\comparison\input\fcTS_test_ssize10_tyear2002_LCfinalmapv6_LCProp2.csv'

In [99]:
X_train = []
y_train = []
ids_train = []
X_test  = []
y_test =[]
ids_test = []

years=range(2001,2003)
ssize=10

for y in years:
    train_x, train_ids, train_y = readSITSData(r'F:\acoca\research\gee\dataset\AMZ\comparison\input\fcTS_train_ssize{}_tyear{}_LCfinalmapv6_LCProp2.csv'.format(ssize,y),y )
    test_x, test_ids, test_y = readSITSData(r'F:\acoca\research\gee\dataset\AMZ\comparison\input\fcTS_train_ssize{}_tyear{}_LCfinalmapv6_LCProp2.csv'.format(ssize,y),y )
    
    X_train.append(train_x)
    y_train.append(train_y)
    ids_train.append(train_ids)

    X_test.append(test_x)
    y_test.append(test_y)
    ids_test.append(test_ids)

target_X = [X_train, X_test]
target_others = [y_train, ids_train, y_test, ids_test]
X_train, X_test = [np.vstack(t) for t in target_X]
y_train, ids_train, y_test, ids_test = [np.array(t).flatten() for t in target_others]

In [43]:
def get_class_idxs(y,N_largest):
    count, bins = np.histogram(y, bins=np.unique(y))
    classidxs_by_size = count.argsort()[::-1]
    return classidxs_by_size[:N_largest]

In [77]:
def filter_largest(X, y, ids, class_idxs):
    print(y.flatten().shape)
    mask = np.isin(y.flatten(), class_idxs)
    y = y.flatten()[mask]
    X = X[mask]
    ids = ids.flatten()[mask]
    return X, y, ids

In [89]:
def get_uniform_idxs(targets, N_uniform=50):
    classes = np.unique(targets)
    idxs = np.array([])
    for c in classes:
        idxs_ = np.argwhere(targets == c)[:N_uniform, 0]
        idxs = np.hstack([idxs, idxs_])
    return idxs.astype(int)

In [88]:
def make_uniform(X, y, ids, N_uniform):
    idxs = get_uniform_idxs(y, N_uniform)
    y = y[idxs]
    X = X[idxs]
    ids = ids[idxs]
    return X, y, ids

In [54]:
classes = os.path.join(r'F:\acoca\research\gee\dataset\AMZ\comparison\input','classes_MCD12Q1v6raw_LCProp2.txt')
with file_io.FileIO(classes, 'r') as f:  # gcp
    classnames = f.readlines()

In [81]:
np.array(classnames)

array(['0|unknown\n', '1|Barren\n', '2|Permanent Snow and Ice\n',
       '3|Water Bodies\n', '4|Urban and Builtup Lands\n',
       '5|Dense Forests\n', '6|Open Forests\n',
       '7|Forest Cropland Mosaics\n', '8|Natural Herbaceous\n',
       '9|Natural Herbaceous Croplands Mosaics\n',
       '10|Herbaceous Croplands\n', '11|Shrublands\n'], dtype='<U39')

In [85]:
class_idxs

array([7], dtype=int64)

In [86]:
class_idxs = get_class_idxs(np.hstack([y_train,y_test]),True)
X, y, ids = filter_largest(X_train, y_train, ids_train, class_idxs)
Xtest, ytest, idstest = filter_largest(X_test, y_test, ids_test, class_idxs)
classnames = np.array(classnames)[class_idxs]

(180,)
(180,)


In [93]:
class_idxs = np.arange(len(classnames))

In [ ]:
# make uniform class distributions
X, y, ids = make_uniform(X, y, ids, 2)
Xtest, ytest, idstest = make_uniform(Xtest, ytest, idstest, 2)

In [102]:
# make uniform class distributions
X, y, ids = make_uniform(X_train, y_train, ids_train, 2)
Xtest, ytest, idstest = make_uniform(X_test, y_test, ids_test, 2)

In [104]:
y.shape

(18,)

In [38]:
from collections import Counter
Counter(y_train.flatten())

Counter({0: 158,
         2: 1000,
         3: 1000,
         4: 1000,
         5: 1000,
         6: 1000,
         7: 1000,
         9: 1000,
         10: 1000})

In [14]:
year=2001
train_file = r'F:\acoca\research\gee\dataset\AMZ\comparison\input\fcTS_train_ssize10_tyear{}_LCfinalmapv6_LCProp2.csv'.format(year)


In [15]:
train_file

'F:\\acoca\\research\\gee\\dataset\\AMZ\\comparison\\input\\fcTS_train_ssize10_tyear2001_LCfinalmapv6_LCProp2.csv'

In [2]:
X_train = [readSITSData(r'F:\acoca\research\gee\dataset\AMZ\comparison\input\fcTS_train_ssize1000_tyear{}_LCfinalmapv6_LCProp2.csv'.format(year), year) for year in range(2001,2003)]


E:\acocac\research\scripts\MTLCC-MODIS-GCP\4_comparison\0_traditional\sits_func.py:79: FutureWarning: read_table is deprecated, use read_csv instead.
  data = pd.read_table(name_file, sep=',', header=0)  # -- one header


In [10]:
np.array(X_train)

ValueError: could not broadcast input array from shape (90,368) into shape (90)

In [6]:
X = [np.concatenate([X[t][1],doy]) for t in range(len(X_train))]

array([[ 743.,  361.,  281., ...,  345.,  353.,  361.],
       [ 975.,  788.,  289., ...,  345.,  353.,  361.],
       [ 723.,  544.,  285., ...,  345.,  353.,  361.],
       ...,
       [ 555., 3680.,  304., ...,  345.,  353.,  361.],
       [ 493., 2897.,  259., ...,  345.,  353.,  361.],
       [ 880., 2852.,  452., ...,  345.,  353.,  361.]])

In [58]:
data = pd.read_table(train_file, sep=',', header=0)  # -- one header

y_data = data['class'] - 1

y = np.asarray(y_data.values, dtype='uint8')

data[ 'array' ] = data[ 'array' ].apply(lambda s: list(literal_eval(s)))
data[ 'array' ] = data[ 'array' ].apply(lambda s: np.concatenate(np.array(s)))

x = np.array(data[ 'array' ]).tolist()

C:\Users\PAPAZ\AppData\Local\Continuum\anaconda2\envs\py36_keras\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  if __name__ == '__main__':


In [5]:
train_file

'F:\\acoca\\research\\gee\\dataset\\AMZ\\comparison\\input\\fcTS_train_ssize10_tyear2001_LCfinalmapv6_LCProp2.csv'